In [1]:
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import json
import pdb
import glob
from nilearn import plotting as nip
from nilearn import image
import pandas as pd

In [2]:
mypath = str(Path().absolute())
labelpath = join(mypath,'MRI_Cloud_Atlases')
dirs = [x[0] for x in os.walk(labelpath)]
dirs = [i for i in dirs if 'git' not in i]
dirs = [i for i in dirs if 'ipynb' not in i]
dirs = [i for i in dirs if '01' in i]
files = []
for d in dirs:
    patient = d.split('/')
    patient = patient[len(patient)-1]
    file = patient+'_FullLabels.nii'
    patFile = os.path.join(d,file)
    files.append(patFile)
files

['/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Pediatric4-8-2f574ca/Ped4-8_01/Ped4-8_01_FullLabels.nii',
 '/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Elderly56/Eld56+_01/Eld56+_01_FullLabels.nii',
 '/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/YoungAdult16-26/YAd16-26_01/YAd16-26_01_FullLabels.nii',
 '/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Adult27-55-e5450fc/Adt27-55_01/Adt27-55_01_FullLabels.nii']

In [3]:
def main():
    for f in files:
        print(f)
        outfile = f.replace('.nii','.json')
        parcel_im = nib.load(f)
        #get_centers(parcel_im,outfile)
        parcel_centers = get_centers(parcel_im,outfile)
        
        with open(outfile) as js:
            js_contents = json.load(js)
            for (k, v) in js_contents.items():
                try:
                    js_contents[k]["center"] = parcel_centers[int(k)]
                except KeyError:
                    js_contents[k]["center"] = None
                    with open(outfile, 'w') as jso:
                        json.dump(js_contents, jso, indent=4)
    return parcel_centers

In [5]:
main()

/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Pediatric4-8-2f574ca/Ped4-8_01/Ped4-8_01_FullLabels.nii
here


MemoryError: 

In [4]:
def get_centers(brain, outfile):
    """
    Get coordinate centers given a nifti image loaded with nibabel
    
    Returns a dictionary of label: coordinate as an [x, y, z] array
    """
    dat = brain.get_data()
    dat = np.squeeze(dat)
    labs = np.unique(dat)
    labs = labs[labs != 0]
    print('here')
    fd_dat = np.stack([np.asarray(dat == lab).astype('float64') for lab in labs], axis=3)
    print('here2')
    parcels = nib.Nifti1Image(dataobj=fd_dat, header=brain.header, affine=brain.affine)
    regions_imgs = image.iter_img(parcels)
    # compute the centers of mass for each ROI
    coords_connectome = [nip.find_xyz_cut_coords(img) for img in regions_imgs]

    
    return dict(zip(labs, coords_connectome))

In [16]:
f = files[0]
parcel_im = nib.load(f)
dat = parcel_im.get_data()
dat = np.squeeze(dat)
labs = np.unique(dat)
labs = labs[labs != 0]
ds = np.asarray(dat == labs[0]).astype('float64')
ds = ds.flatten()
sum(ds)

17120.0

In [19]:
def get_sizes(brain, outfile):
    """
    Get coordinate centers given a nifti image loaded with nibabel
    
    Returns a dictionary of label: coordinate as an [x, y, z] array
    """
    dat = brain.get_data()
    dat = np.squeeze(dat)
    labs = np.unique(dat)
    labs = labs[labs != 0]
    sizes = []
    for lab in range(0,len(labs)):
        ds = np.asarray(dat == labs[lab]).astype('float64')
        ds = ds.flatten()
        sizes.append(sum(ds))

    
    return sizes

In [23]:
for f in files:
        print(f)
        outfile = f.replace('.nii','.json')
        parcel_im = nib.load(f)
        #get_centers(parcel_im,outfile)
        sizes = get_sizes(parcel_im,outfile)
        createJSON(sizes,outfile)
        with open(outfile) as f:
            data = json.load(f)
            print(type(data))
    
        for i in range(0,len(sizes)):
            data[str(i+1)]["size"] = str(sizes[i])
        
        with open(outfile, 'w') as f:
            json.dump(data, f, indent=4, sort_keys=True)
        

/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Pediatric4-8-2f574ca/Ped4-8_01/Ped4-8_01_FullLabels.nii
<class 'dict'>
<class 'dict'>
/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Elderly56/Eld56+_01/Eld56+_01_FullLabels.nii
<class 'dict'>
<class 'dict'>
/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/YoungAdult16-26/YAd16-26_01/YAd16-26_01_FullLabels.nii
<class 'dict'>
<class 'dict'>
/home/pmyers16/Desktop/neuroparc/atlases/MRI_Cloud_Atlases/Adult27-55-e5450fc/Adt27-55_01/Adt27-55_01_FullLabels.nii
<class 'dict'>
<class 'dict'>


In [21]:
def createJSON(sizes, outname):
    data = {}
    data['0'] = {}
    data['0']['label'] = 'empty'
    print(type(data))
    for row in range(0,len(sizes)):
        data[str(row+1)]={}
    with open(outname, 'w') as outfile:  
        json.dump(data, outfile, indent=4, sort_keys=True)